# Working with the hourly Global Drifter Program (GDP) data

In this Notebook, we use the [hourly data](https://www.aoml.noaa.gov/phod/gdp/hourly_data.php) from the [NOAA Global Drifter Program](https://www.aoml.noaa.gov/global-drifter-program/) to illustrate the steps to preprocess a Lagrangian dataset into a `RaggedArray` format that can be then emitted as `xarray.Dataset` or `awkward.Array` instances.

## Import the `RaggedArray` class

In [ ]:
from clouddrift import RaggedArray

The `RaggedArray` class can be initialized with a series of `dict`s matching the name of the variables of the original data with the ragged representation of the data and their respectives attributes.

In [ ]:
help(RaggedArray)

The length of the variables in the dataset is either equal to the number of trajectories (`nb_traj`) or the number of observations (`obs`) derived along the trajectories. 

The first three dictionnaries, `coords`, `metadata`, and `data`, match the variable names for coordinates variables, metatadata variables, and data variables, respectively, with the ragged representation of the data.

- The coordinates are mandatory variables (length `obs`) for the ragged array to be used with the library and are always `time`, `lon`, `lat`, and `ids`. 
- The metadata variables (length `nb_traj`) are constant values associated with a single trajectory such as the length of the observations (`rowsize`), the deployment information (`deploy_lon`, `deploy_lat`, `deploy_date`), the type of buoys (`typebuoy`), etc.
- The data variables (length `obs`) are quantity derived along the trajectories such as velocity component (`ve`, `vn`), the sea surface temperature and its uncertainty (`sst`, `err_sst`), the drogue presence flag (`drogue_status`), etc.

The last two dictionaries, `attrs_global` and `attrs_variables`, are optional and contain the attributes related to the dataset and each variable, respectively.

## The `RaggedArray.from_files()` method

This method is available to initialize a `RaggedArray` instance from a series of files.

In [ ]:
help(RaggedArray.from_files)

This method was inspired by the [Pangeo Forge](https://pangeo-forge.readthedocs.io/en/latest/) project which aims at easing the extraction of data from traditional data archives and deposition in cloud object storage. For our `RaggedArray.from_files()` method, the different parameters are:

- `indices`: a list of indices (or identification numbers) that will be iterated over to concatenate the files into the `RaggedArray` instance;
- `preprocess_func`: A preprocessing function with the following signature:
    - `Signature: preprocess_func(index: int) -> xarray.core.dataset.Dataset`, where the index parameter is an identifier of a trajectory, e.g. the identification number of an Argo float) and returns an [xarray Dataset](https://docs.xarray.dev/en/latest/generated/xarray.Dataset.html). 
- `vars_coords`: a dictionary mapping the mandatory coordinates list to the name of those variables in the dataset, e.g.
    `coords = {'ids': 'number', 'time': 't', 'lon': 'longitude', 'lat': 'latitude'}`
- `vars_meta`: an optional list of variable names containing metadata information about the trajectory (size: 1 per trajectory)
- `vars_data`: an optional list of variable names containing the data along the trajectory (size: number of observations per trajectory)
- `rowsize_func`: an optional function that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)

Because every dataset is unique, the preprocessing function is used to perform operations such as: formatting the date and time, changing the type of the variables, modifying the metadata, and similar. The class also needs to initially calculate the sum of all observations to allocate memory. To speed up this process where a lot of preprocessing is needed, it is possible to provide a second function `rowsize_func`, that returns directly the number of observations of a trajectory (`Signature: rowsize_func(index: int) -> int`). By default, this operation is performed using `lambda i: preprocess_func(i).dims['obs']`. 

We provide preprocessing function for different datasets in the `data/` folder (`gdp.py`, `gdp6h.py`, `parcels.py`, etc.) and those can serve as a guide to defined a new set of functions for another dataset.

# Dataset-specific functions
The `gdp.py` module contains a number of specific functions for the current GDP files, including:
- `gdp.preprocess`: applies preprocessing routine and returned a `xarray.Dataset` for a specific trajectory 
- `gdp.download`: fetches NetCDF files from the GDP FTP server
- `gdp.rowsize [Optional]`: returns the dimension of a specific trajectory to speed up the preprocessing

In [ ]:
import sys
sys.path.append('../')
from data import gdp

## Download the GDP data

The `gdp.download` function will store the raw dataset into the `data/raw/gdp-v2.00/` folder (specified in the `gdp.py` module). By default `download()` will download the complete GPD dataset (containing 17,324 files for versions 1.04c and 2.00) from the AOML repository ([link](https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/hourly/v2.00/netcdf/)).

In [ ]:
help(gdp.download)

With this function, it is also possible to retrieve a subset from a `drifter_ids` list or specified an integer `n_random_id` to randomly retrieve `n` trajectories. If both arguments are given, the function downloads `n_random_id` out of the list `drifter_ids`. The function returns the list of `drifters_ids` that was downloaded, and can be passed to create the ragged array.

In [ ]:
drifter_ids = gdp.download(n_random_id=100)

In [ ]:
drifter_ids[:5]

## Create the `RaggedArray` instance

Once the data are downloaded, the *RaggedArray* object can be created and archived as a NetCDF file or a parquet file. In addition, the ragged array can be converted to an [xarray Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) or an [Awkward Array](https://awkward-array.readthedocs.io/en/stable/_auto/ak.Array.html) in order to conduct analyses.

Before we do that, we must first define the dataset-specific coordinates, data, and metadata information.

In [ ]:
coords = {
    'ids': 'ids',
    'time': 'time',
    'lon': 'longitude', # or alternatively lon360
    'lat': 'latitude'
}
metadata = [
    'ID',
    'rowsize', 
    'WMO',
    'expno',
    'deploy_date',
    'deploy_lat',
    'deploy_lon',
    'end_date',
    'end_lat',
    'end_lon',
    'drogue_lost_date',
    'typedeath',
    'typebuoy',
    'location_type',
    'DeployingShip',
    'DeploymentStatus',
    'BuoyTypeManufacturer',
    'BuoyTypeSensorArray',
    'CurrentProgram',
    'PurchaserFunding',
    'SensorUpgrade',
    'Transmissions',
    'DeployingCountry',
    'DeploymentComments',
    'ManufactureYear',
    'ManufactureMonth',
    'ManufactureSensorType',
    'ManufactureVoltage',
    'FloatDiameter',
    'SubsfcFloatPresence',
    'DrogueType',
    'DrogueLength',
    'DrogueBallast',
    'DragAreaAboveDrogue',
    'DragAreaOfDrogue',
    'DragAreaRatio',
    'DrogueCenterDepth',
    'DrogueDetectSensor'
]
data = [
    've',
    'vn',
    'err_lat',
    'err_lon',
    'err_ve',
    'err_vn',
    'gap',
    'sst',
    'sst1',
    'sst2',
    'err_sst',
    'err_sst1',
    'err_sst2',
    'flg_sst',
    'flg_sst1',
    'flg_sst2',
    'drogue_status'
]

In [ ]:
ra = RaggedArray.from_files(
    drifter_ids,
    gdp.preprocess,
    coords, 
    metadata,
    data,
    rowsize_func=gdp.rowsize
)

## Export to data files

Currently, the library supports exporting the `RaggedArray` instance to NetCDF and Apache Parquet file formats.

### NetCDF

In [ ]:
ra.to_netcdf('../data/process/gdp_v2.00.nc')

### Apache Parquet

In [ ]:
ra.to_parquet('../data/process/gdp_v2.00.parquet')

## Import from data files

As with exporting to files, we can import a `RaggedArray` instance from a NetCDF or Apache Parquet files. 

### NetCDF

In [ ]:
ra = RaggedArray.from_netcdf('../data/process/gdp_v2.00.nc')

### Apache Parquet

In [ ]:
ra = RaggedArray.from_parquet('../data/process/gdp_v2.00.parquet')

## Converting to data structures for analysis

Typically, you would not do your data analysis directly on a `RaggedArray` instance.
Instead, you first convert it to a data structure that is geared for data analysis and exploratory science.
Currently, the library supports Xarray Dataset and Awkward Array structures.

### Xarray Dataset

In [ ]:
ds = ra.to_xarray()

Now we have our GDP dataset as an Xarray Dataset while also being optimized for memory.
Let's see what's inside:

In [ ]:
ds


Time, longitude, and latitude arrays are 1-dimensional and are effectively a concatenated list of drifter trajectories. The `ids` array can be used to mask (subset) a specific drifter. Let's make a trajectory map by looping over all the trajectories and plotting them one by one.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for drifter_id in drifter_ids:
    this_drifter = ds.ids == drifter_id
    plt.plot(ds.lon[this_drifter], ds.lat[this_drifter], linestyle='', marker='.', ms=0.1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('100 random trajectories from the hourly GDP dataset')

You may find that the method above is relatively slow to plot for a large number of trajectories. Alternatively, we can construct an index variable that indexes the beginning of each trajectory in the ragged arrays (with index `0` for the first trajectory). We first need to import the `numpy` package to conduct some mathematical operations to calculate that index variable.

In [ ]:
import numpy as np

In [ ]:
traj_idx = np.insert(np.cumsum(ds.rowsize.values),0,0)

In [ ]:
for j in range(len(drifter_ids)):
    plt.plot(ds.lon[slice(traj_idx[j],traj_idx[j+1])], ds.lat[slice(traj_idx[j],traj_idx[j+1])], linestyle='', marker='.', ms=0.1)    
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('100 random trajectories from the hourly GDP dataset')

### Awkward Array

In [ ]:
ds = ra.to_awkward()

With an awkward array, there is no need to define an index variable as for an xarray Dataset:

In [ ]:
for n in range(len(drifter_ids)):
    plt.plot(ds.obs['lon'][n], ds.obs['lat'][n], linestyle='', marker='.', ms=0.1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('100 random trajectories from the hourly GDP dataset')